In [33]:
class TransVal()
case class Transaction(assignedVar: TransVal, readVars: List[TransVal]) {
  def isDependentOn(other: Transaction): Boolean = {
    other.readVars.contains(this.assignedVar) ||
      this.readVars.contains(other.assignedVar) ||
      this.assignedVar == other.assignedVar
  }

  def isIndependentOn(other: Transaction): Boolean = !isDependentOn(other)
}

val X = new TransVal()
val Y = new TransVal()
val Z = new TransVal()
val V = new TransVal()

val A = Transaction(X, List(X, Y))
val B = Transaction(Y, List(Y, Z))
val C = Transaction(X, List(X, Z))
val D = Transaction(Z, List(Y, Z))

val ops = Array(A, B, C, D)

val word = Array(B,A,A,D,C,B)

defined class TransVal
defined class Transaction
X: $user.TransVal = cmd28$$user$TransVal@4b47601e
Y: $user.TransVal = cmd28$$user$TransVal@864448c
Z: $user.TransVal = cmd28$$user$TransVal@4b02c8d1
V: $user.TransVal = cmd28$$user$TransVal@745199b1
A: $user.Transaction = Transaction(
  cmd28$$user$TransVal@4b47601e,
  List(cmd28$$user$TransVal@4b47601e, cmd28$$user$TransVal@864448c)
)
B: $user.Transaction = Transaction(
  cmd28$$user$TransVal@864448c,
  List(cmd28$$user$TransVal@864448c, cmd28$$user$TransVal@4b02c8d1)
)
C: $user.Transaction = Transaction(
  cmd28$$user$TransVal@4b47601e,
  List(cmd28$$user$TransVal@4b47601e, cmd28$$user$TransVal@4b02c8d1)
)
D: $user.Transaction = Transaction(
  cmd28$$user$TransVal@4b02c8d1,
  List(cmd28$$user$TransVal@864448c, cmd28$$user$TransVal@4b02c8d1)
)
ops: Array[$user.Transaction] = Array(
  Transaction(
    cmd28$$user$TransVal@4b47601e,
    List(cmd28$$user$TransVal@4b47601e, cmd28$$user$TransVal@864448c)
  ),
  Transaction(
    cmd28$$user$T

In [34]:
val D = ops.map(i => ops.map(o => i.isDependentOn(o)))

D: Array[Array[Boolean]] = Array(
  Array(true, true, true, false),
  Array(true, true, false, true),
  Array(true, false, true, true),
  Array(false, true, true, true)
)

In [35]:
val I = ops.map(i => ops.map(o => i.isIndependentOn(o)))

I: Array[Array[Boolean]] = Array(
  Array(false, false, false, true),
  Array(false, false, true, false),
  Array(false, true, false, false),
  Array(true, false, false, false)
)

In [36]:
val m = new TransVal()

m: TransVal = cmd28$$user$TransVal@1ee80977

In [37]:
class Step(val t: Transaction, val i: Integer)

defined class Step

In [38]:

val ww = word.zipWithIndex.map(x => new Step(x._1, x._2))


ww: Array[Step] = Array(
  cmd32$$user$Step@6dea632e,
  cmd32$$user$Step@17dab7ed,
  cmd32$$user$Step@7e2e6e57,
  cmd32$$user$Step@7c3b92e4,
  cmd32$$user$Step@42f67d5e,
  cmd32$$user$Step@51cf32
)

In [39]:
val nonqueue = ww.map(w => (w,0))

nonqueue: Array[(Step, Int)] = Array(
  (cmd32$$user$Step@6dea632e, 0),
  (cmd32$$user$Step@17dab7ed, 0),
  (cmd32$$user$Step@7e2e6e57, 0),
  (cmd32$$user$Step@7c3b92e4, 0),
  (cmd32$$user$Step@42f67d5e, 0),
  (cmd32$$user$Step@51cf32, 0)
)

In [40]:
def appotife(ar:Array[(Step, Int)], v: Step) : Array[(Step, Int)] = {
    ar.map(o => o match {
        case o if (o._1.i < v.i) => o
        case o if o._1 == v  => o
        case o if o._1.t.isDependentOn(v.t) => (o._1, o._2 + 1)
        case o => o
    })
}

defined function appotife

In [42]:
val qued = ww.foldLeft(nonqueue)((acc, i) => appotife(acc, i))

qued: Array[(Step, Int)] = Array(
  (cmd32$$user$Step@6dea632e, 0),
  (cmd32$$user$Step@17dab7ed, 1),
  (cmd32$$user$Step@7e2e6e57, 2),
  (cmd32$$user$Step@7c3b92e4, 1),
  (cmd32$$user$Step@42f67d5e, 3),
  (cmd32$$user$Step@51cf32, 4)
)

In [44]:
def deporique(ar:Array[(Step, Int)]) : (Array[Step], Array[(Step, Int)]) = {
    val fl = ar.filter(o => o._2 == 0).map(g => g._1)
    val ot = ar.filter(o => o._2 > 0)
    val ott = ot.map(o => (o._1, o._2 - fl.count(f => f.t.isDependentOn(o._1.t) && f.i < o._1.i)))
    (fl, ott)
}

defined function deporique

In [45]:
def veriquev(a : Array[(Step, Int)], ff: Array[Array[Step]]):Array[Array[Step]] = {
    if (a.length == 0) {
        //sorry
        return ff
    } else {
        val h = deporique(a)
        return veriquev(h._2, ff:+h._1)
    }
}



defined function veriquev

In [46]:
val y = veriquev(qued, Array.empty[Array[Step]])

y: Array[Array[Step]] = Array(
  Array(cmd32$$user$Step@6dea632e),
  Array(cmd32$$user$Step@17dab7ed, cmd32$$user$Step@7c3b92e4),
  Array(cmd32$$user$Step@7e2e6e57),
  Array(cmd32$$user$Step@42f67d5e, cmd32$$user$Step@51cf32)
)

In [47]:
y.map(u => u.map(s => s.i))
// Yay

res41: Array[Array[Integer]] = Array(Array(0), Array(1, 3), Array(2), Array(4, 5))

In [48]:
val deps = ww.map(w => ww.map(o => w.t.isDependentOn(o.t) && w.i > o.i))

deps: Array[Array[Boolean]] = Array(
  Array(false, false, false, false, false, false),
  Array(true, false, false, false, false, false),
  Array(true, true, false, false, false, false),
  Array(true, false, false, false, false, false),
  Array(false, true, true, true, false, false),
  Array(true, true, true, true, false, false)
)

In [49]:
import scala.collection.mutable.ArrayBuffer

var isAnythingChanged : Boolean = true
val D2 = ArrayBuffer[ArrayBuffer[Boolean]]()
for (y1 <- deps.indices) {
    D2 += ArrayBuffer[Boolean]()
    for (x1 <- deps(y1).indices) {
        D2(y1) += false;
    }
}



while (isAnythingChanged) {
    val cp = D2.map(o => o.map(x => x))
    for (y1 <- deps.indices) {
        for (x1 <- deps(y1).indices) {
            for (w <- deps.indices) {
                D2(y1)(x1) = D2(y1)(x1) || (deps(y1)(w) && deps(w)(x1) && y1 > w && w > x1)
            }
        }
    }
    
    isAnythingChanged = false
    
    for (y1 <- deps.indices) {
        for (x1 <- deps(y1).indices) {
            isAnythingChanged = isAnythingChanged || D2(y1)(x1) != cp(y1)(x1)
        }
    }
}



import scala.collection.mutable.ArrayBuffer
isAnythingChanged: Boolean = false
D2: ArrayBuffer[ArrayBuffer[Boolean]] = ArrayBuffer(
  ArrayBuffer(false, false, false, false, false, false),
  ArrayBuffer(false, false, false, false, false, false),
  ArrayBuffer(true, false, false, false, false, false),
  ArrayBuffer(false, false, false, false, false, false),
  ArrayBuffer(true, true, false, false, false, false),
  ArrayBuffer(true, true, false, false, false, false)
)

In [50]:
val indirDeps = ArrayBuffer[ArrayBuffer[Boolean]]()
for (y1 <- deps.indices) {
    indirDeps += ArrayBuffer[Boolean]()
    for (x1 <- deps(y1).indices) {
        indirDeps(y1) += false;
    }
}

for (y1 <- deps.indices) {
    for (x1 <- deps(y1).indices) {
        indirDeps(y1)(x1) = deps(y1)(x1) && !D2(y1)(x1)
    }
}

indirDeps: ArrayBuffer[ArrayBuffer[Boolean]] = ArrayBuffer(
  ArrayBuffer(false, false, false, false, false, false),
  ArrayBuffer(true, false, false, false, false, false),
  ArrayBuffer(false, true, false, false, false, false),
  ArrayBuffer(true, false, false, false, false, false),
  ArrayBuffer(false, false, true, true, false, false),
  ArrayBuffer(false, false, true, true, false, false)
)

In [51]:
println("http://www.webgraphviz.com/")
println("digraph G {")
for (y1 <- indirDeps.indices) {
    for (x1 <- indirDeps(y1).indices) {
        if (indirDeps(y1)(x1)) {
            println("  " + x1 + " -> " + y1)
        }
    }
}
println("}")

http://www.webgraphviz.com/
digraph G {
  0 -> 1
  1 -> 2
  0 -> 3
  2 -> 4
  3 -> 4
  2 -> 5
  3 -> 5
}


In [53]:
var pardeps = ArrayBuffer[(Int, Int)]()
for (y1 <- indirDeps.indices) {
    for (x1 <- indirDeps(y1).indices) {
        if (indirDeps(y1)(x1)) {
            pardeps += ((x1, y1))
        }
    }
}

var wds = ww.map(w => w.i)

val printed = ArrayBuffer[Int]()

var doItOnceMore = true

while (pardeps.length != 0 || doItOnceMore){
    if (pardeps.length == 0) {
        doItOnceMore = false
    }
    var floatable = wds.filter(w => pardeps.count(p => p._2 == w) == 0 && !printed.contains(w))
    println("")
    print("[")
    for (y1 <- floatable) {
        printed += y1;
        print(y1 + " ")
    }
    print("]")
    println("")
    pardeps = pardeps.filter(p => floatable.count(w => w == p._1) == 0)
}



[0 ]

[1 3 ]

[2 ]

[4 5 ]


pardeps: ArrayBuffer[(Int, Int)] = ArrayBuffer()
wds: Array[Integer] = Array(0, 1, 2, 3, 4, 5)
printed: ArrayBuffer[Int] = ArrayBuffer(0, 1, 3, 2, 4, 5)
doItOnceMore: Boolean = false